In [1]:
import os
import functions as fun
from geopy import distance
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
from dotenv import load_dotenv
import pandas as pd

In [2]:
Cambridge = 42.3731956,-71.1198561

In [3]:
sub_CA = pd.read_csv("../Data/sub_CA.csv")


# Cambridge (USA)

In [4]:
load_dotenv()

True

In [5]:
token_foursquare = os.getenv("Token_foursquare")

- First we look for Starbucks close to the office and clean the results keeping only the columns that we are interested about

In [6]:
starbucks_CA = fun.get_results("starbucks", Cambridge, 10, 1000, token_foursquare)
df_CA = fun.clean_results_first(starbucks_CA, "Coffee")

- We look for preschools or kindergarten around the office and clean the results keeping only the columns that we are interested about

In [7]:
kinder_CA = fun.get_results("preschool", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results(kinder_CA, df_CA, "Preschool")

- We look for nightclubs around the office and clean the results keeping only the columns that we are interested about

In [8]:
club_CA = fun.get_results("night club", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results(club_CA, df_CA, "Club")

- We look for vegan restaurants around the office and clean the results keeping only the columns that we are interested about

In [9]:
vegan_CA = fun.get_results("vegan", Cambridge, 10, 3000, token_foursquare)
df_CA = fun.clean_results(vegan_CA, df_CA, "Vegan restaurant")

- We look for basketball arena (Chase Center) around the office and clean the results keeping only the columns that we are interested about

In [10]:
Basketball_CA = fun.get_results("TD Garden", Cambridge, 1, 5000, token_foursquare)
df_CA = fun.clean_results(Basketball_CA, df_CA, "Basketball arena")

- We look for dog hairdressers around the office and clean the results keeping only the columns that we are interested about

In [11]:
pet_CA = fun.get_results("dog grooming", Cambridge, 5, 5000, token_foursquare)
df_CA = fun.clean_results_lo(pet_CA, df_CA, "Pet hairdresser")

- We look for the closest Airport to office and clean the results keeping only the columns that we are interested about

In [12]:
Airports_CA = fun.get_results_airport("Airport", Cambridge, 2, 50000, token_foursquare)
df_CA = fun.clean_results_lo(Airports_CA, df_CA, "Airport")

- We add the data of the new office that we have selected

In [13]:
df_CA = fun.add_company(df_CA, "5, John F. Kennedy St, Cambridge, MA 02138", "West Cambridge", 42.3731956,-71.1198561)

- We create a new row with the distance from the office to the diferent places

In [14]:
df_CA["Distance"] = df_CA.apply(fun.calc_distance_CA, axis = 1).round(2)

In [15]:
df_CA

,name,address,neighborhood,lat,lon,category,Distance
0,Company,"5, John F. Kennedy St, Cambridge, MA 02138",West Cambridge,42.373196,-71.119856,NaN,0.00
1,Starbucks,"468 Broadway (at Broadway Marketplace), Cambri...",Central Square,42.373858,-71.112954,Coffee,573.23
2,Starbucks,"1662 Massachusetts Ave (at Shepard St), Cambri...",Central Square,42.382029,-71.119939,Coffee,981.24
3,Starbucks Reserve,1380 Massachusetts Ave (btwn JFK & Dunster St)...,Allston,42.373358,-71.119320,Coffee,47.70
4,Starbucks,"1662 Massachusetts Ave (at Shepard St), Cambri...",Central Square,42.382029,-71.119939,Coffee,981.24
5,Agassiz Preschool,"1803 Massachusetts Ave (Rose), Cambridge, MA 0...",Avon Hill,42.386695,-71.118798,Preschool,1502.05
6,Agassiz Preschool,"184 Summer St (Spring St), Somerville, MA 02143",Central Square,42.386926,-71.108152,Preschool,1804.24
7,King Open Extended Day,"870 Cambridge St, Cambridge, MA 02141",East Somerville,42.372329,-71.091326,Preschool,2351.87
8,KinderCare Learning Center,"18 Holland St, Somerville, MA 02144",Avon Hill,42.397394,-71.123566,Preschool,2705.28
9,Kiddie Academy of Cambridge,"299 Prospect St, Cambridge, MA 02139",East Somerville,42.372829,-71.098869,Preschool,1729.08


In [23]:
df_CA.groupby("category")["Distance"].describe()

,count,mean,std,min,25%,50%,75%,max
category,,,,,,,,
Airport,1.0,8139.8600,NaN,8139.86,8139.8600,8139.860,8139.8600,8139.86
Basketball arena,1.0,4810.3200,NaN,4810.32,4810.3200,4810.320,4810.3200,4810.32
Club,10.0,1706.0000,910.132729,91.13,1094.9425,1984.940,2317.3525,2773.79
Coffee,4.0,645.8525,442.730163,47.70,441.8475,777.235,981.2400,981.24
Preschool,8.0,2274.5225,533.395861,1502.05,1785.4500,2423.365,2634.8025,2997.49
Vegan restaurant,10.0,1044.2970,1031.782501,45.96,182.5500,668.005,1860.5075,2910.43


In [16]:
CA_map = Map(location = Cambridge, zoom_start = 12)

In [17]:
for index, row in df_CA.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["lat"], row["lon"]], "tooltip": row["name"]}
    
    #2. Icon
    if row["category"] == "Coffee":        
        icon = Icon (
            color="white",
            opacity = 0.6,
            prefix = "fa",
            icon="coffee",
            icon_color = "black"
        )
    elif row["category"] == "Preschool":
        icon = Icon (
            color="pink",
            opacity = 0.6,
            prefix = "fa",
            icon="graduation-cap",
            icon_color = "yellow"
        )
    elif row["category"] == "Club":
        icon = Icon (
            color="darkpurple",
            opacity = 0.6,
            prefix = "fa",
            icon="glass",
            icon_color = "white"
        )
    elif row["category"] == "Vegan restaurant":
        icon = Icon (
            color="green",
            opacity = 0.6,
            prefix = "fa",
            icon="cutlery",
            icon_color = "white"
        )
    elif row["category"] == "Basketball arena":
        icon = Icon (
            color="orange",
            opacity = 0.6,
            prefix = "fa",
            icon="futbol-o",
            icon_color = "white"
        )
    elif row["category"] == "Airport":
        icon = Icon (
            color="blue",
            opacity = 0.6,
            prefix = "fa",
            icon="plane",
            icon_color = "white"
            
        )
    elif row["category"] == "Pet hairdresser":
        icon = Icon (
            color="red",
            opacity = 0.6,
            prefix = "fa",
            icon="scissors",
            icon_color = "white"
        )
    else:
        row["category"] == "Company",
        icon = Icon (
        color="black",
        opacity = 0.9,
        prefix = "fa",
        icon = "briefcase",
        icon_color = "white",
        icon_size=(30, 30)
        )
            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(CA_map)

In [18]:
for index, row in sub_CA.iterrows():
    
    #1. MARKER without icon
    district = {"location": [row["Lat"], row["Long"]], "tooltip": row["name"]}
    
    #2. Icon       
    icon = Icon (
    color="white",
    opacity = 0.6,
    prefix = "fa",
    icon="building",
    icon_color = "black"
        )

            
    #3. Marker
    marker = Marker(**district, icon = icon, radius = 2)
    
    #4. Add the Marker
    marker.add_to(CA_map)

In [19]:
CA_map

In [20]:
CA_map.save('../images/Cambridge_map.html')